### <div align="center">***API REQUEST FOR DATABASE***</div>
***
Extracción de los registros relativos a los distritos 7,8,9: **Chamberí**, **Fuencarral-El Pardo**, **Moncloa-Aravaca**.

In [5]:
## CREDENCIALES PARA PODER USAR LA API

api_key= 

### **IMPORTS**

In [6]:
from yelpapi import YelpAPI
import pandas as pd
import requests
import folium
import json

yelp_api = YelpAPI(api_key)

from utils import distritos

## CHAMBERI (6) : FUENCARRAL (7) : MONCLOA (8)
LATITUD = distritos[0]["latitude"]
LONGITUD = distritos[0]["longitude"]
RADIO = 1000

### **EXTRACCION CON LA API**

In [7]:
## CONFIGURACIÓN PARA QUE PANDAS MUESTRE LAS COLUMNAS DEL DF SIN TRUNCARLAS
pd.set_option('display.max_columns', None) 

In [8]:
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
    'Authorization': 'Bearer ' + api_key
}

res = []
for dis in range(1):
    params = {    
    "latitude": LATITUD,  # Latitud del epicentro de búsqueda
    "longitude": LONGITUD,  # Longitud del epicentro de búsqueda
    "radius": RADIO,  # Radio en metros para buscar restaurantes alrededor de la ubicación
    'categories': 'restaurants',  # Seleccionamos la categoría de comercio a extraer
    'limit': 50,  # Número de resultados que recupera (max 50
    }

    response = requests.get(url, headers=headers, params=params) # GET request
    if response.headers.get("Content-Type") == "application/json":
        try:
            # Parse the JSON response
            data = response.json()

            # Extract and process restaurants
            restaurants = data['businesses']        
                
            for restaurant in restaurants:
                description = ""
                id = restaurant['id']
                name = restaurant["name"]
                categories = restaurant['categories']
                rating = restaurant['rating']    
                address = ', '.join(restaurant['location']['display_address'])
                
                ## ALIAS
                if "alias" in restaurant:
                    alias = restaurant["alias"]
                else:
                    alias = None

                ## URL IMAGEN
                if "image_url" in restaurant:
                    image_url = restaurant["image_url"]
                else:
                    image_url = None

                ## URL NEGOCIO
                if "url" in restaurant:
                    url = restaurant["url"]
                else:
                    url = None

                ## CLOSED
                if "is_closed" in restaurant:
                    is_closed = restaurant["is_closed"]
                else:
                    is_closed = None

                ## REVIEW_COUNT
                if "review_count" in restaurant:
                    review_count = restaurant["review_count"]
                else:
                    review_count = None
                
                ## COORDINATES
                if "coordinates" in restaurant:
                    latitude = restaurant["coordinates"]["latitude"]
                    longitude = restaurant["coordinates"]["longitude"]
                else:
                    latitude = None
                    longitude = None
                
                ## LOCATION
                if "location" in restaurant:
                    location = restaurant["location"]
                else:
                    location = None

                ## PHONE
                if "display_phone" in restaurant:
                    display_phone = restaurant["display_phone"]
                else:
                    display_phone = None

                ## PRICE
                if "price" in restaurant:
                    price = restaurant["price"]
                else:
                    price = None 

                ## HOURS   
                if "hours" in restaurant:
                    hours = restaurant["hours"]
                else:
                    hours = None
                
                ## TRANSACTIONS
                if "transactions" in restaurant:
                    transactions = restaurant["transactions"]
                else:
                    transactions = None

                ## LANGUAGES
                if "languages" in restaurant:
                    languages = restaurant["languages"]
                else:
                    languages = None
                
                ## ATRIBUTES
                if "attributes" in restaurant:
                    attributes = restaurant["attributes"]
                else:
                    attributes = None

                ## TRANSIT
                if "transit" in restaurant:
                    transit = restaurant["transit"]
                else:
                    transit = None

                ## BEST TIME
                if "best_time" in restaurant:
                    best_time = restaurant["best_time"]
                else:
                    best_time = None

                ## DELIVERY
                if "delivery" in restaurant:
                    delivery = restaurant["delivery"]
                else:
                    delivery = None

                ## PICKUP
                if "pickup" in restaurant:
                    pickup = restaurant["pickup"]
                else:
                    pickup = None

                ## ONLINE ORDER
                if "online_ordering" in restaurant:
                    online_ordering = restaurant["online_ordering"]
                else:
                    online_ordering = None
                        
                ## PRIMEROS 5 REVIEWS 
                for c in categories:
                    description = description + c["title"] + " "
                description = description  

                # Make the API call to retrieve reviews for the current business
                reviews_response = yelp_api.reviews_query(id=restaurant["id"],location="Madrid", lang="es")
                        
                # Extract the reviews from the response
                reviews = reviews_response['reviews']
                        
                # Process and print the reviews
                        
                rev = ""
                for review in reviews[:5]:
                    review_text = review['text']
                    rating = review['rating']
                    reviewer_name = review['user']['name']

                    rev = rev + review_text + ". "   
                        
                
                res.append({
                        "id" : id,
                        "name" : name, 
                        "rating": rating, 
                        "address": address, 
                        "description": description, 
                        "review": rev,
                        "alias":alias,
                        "name":name,
                        "image_url":image_url,
                        "is_closed":is_closed,
                        "review_count":review_count,
                        "latitude":latitude,
                        "longitude": longitude,
                        "location":location,
                        "display_phone":display_phone,
                        "price":price,
                        "hours":hours,
                        "transactions":transactions,
                        "languages":languages,
                        "attributes":attributes,
                        "transit":transit,
                        "best_time":best_time,
                        "delivery":delivery,
                        "pickup":pickup,
                        "online_ordering":online_ordering
                })
        except json.JSONDecodeError:
            print("La respuesta no es un JSON válido")

df_distrito = pd.DataFrame(res)

In [9]:
df_distrito

,id,name,rating,address,description,review,alias,image_url,is_closed,review_count,latitude,longitude,location,display_phone,price,hours,transactions,languages,attributes,transit,best_time,delivery,pickup,online_ordering
0,m-suxON2HwsWYImvTNX1Jw,Botín,3,"Calle de Cuchilleros, 17, 28005 Madrid, Spain",Spanish Steakhouses,Historical restaurant that we felt honored to ...,botín-madrid-3,https://s3-media2.fl.yelpcdn.com/bphoto/UpGNFa...,False,648,40.414060,-3.708030,"{'address1': 'Calle de Cuchilleros, 17', 'addr...",+34 913 66 42 17,€€€,None,[],None,None,None,None,None,None,None
1,ql9zlnKW4h_efrketA3Mrg,Chocolatería San Ginés,4,"Pasadizo de San Ginés, 5, 28013 Madrid, Spain",Chocolatiers & Shops Churros Breakfast & Brunch,"Before arriving in Madrid this March, the last...",chocolatería-san-ginés-madrid-6,https://s3-media4.fl.yelpcdn.com/bphoto/hwpN-t...,False,1333,40.416760,-3.706800,"{'address1': 'Pasadizo de San Ginés, 5', 'addr...",+34 913 65 65 46,€,None,[],None,None,None,None,None,None,None
2,ZIYLcIOrhGGHt-YoWFj_Iw,El Mesón del Champiñón,3,"Calle Cava de San Miguel, 17, 28005 Madrid, Spain",Spanish Tapas Bars,THE BIG: Best mushroom tapas place in Madrid!\...,el-mesón-del-champiñón-madrid-3,https://s3-media3.fl.yelpcdn.com/bphoto/Mi51ST...,False,102,40.415006,-3.708161,"{'address1': 'Calle Cava de San Miguel, 17', '...",+34 915 59 67 90,€€,None,[],None,None,None,None,None,None,None
3,uHL7ravKYyrTl07fv_hfUg,Rosi La Loca,4,"Calle de Cádiz, 4, 28012 Madrid, Spain",Tapas Bars Spanish,Rosie La Loca is the result of someone saying ...,rosi-la-loca-madrid,https://s3-media3.fl.yelpcdn.com/bphoto/SN6sHZ...,False,194,40.415814,-3.702979,"{'address1': 'Calle de Cádiz, 4', 'address2': ...",+34 915 32 66 81,€€€,None,[],None,None,None,None,None,None,None
4,Hn7-q6opT1g7ZJoZ9ZpjzA,Metro Bistro,5,"Calle Imperial, 3, 28012 Madrid, Spain",Mediterranean Spanish Signature Cuisine,I highly recommend metro bistro because of the...,metro-bistro-madrid-5,https://s3-media2.fl.yelpcdn.com/bphoto/uDgvjX...,False,34,40.414641,-3.706778,"{'address1': 'Calle Imperial, 3', 'address2': ...",+34 913 66 33 19,€€,None,[],None,None,None,None,None,None,None
5,S-Md-BF6C53iB2OlB5BCww,Casa Lucas,5,"Calle de la Cava Baja, 30, 28005 Madrid, Spain",Tapas Bars Wine Bars,Great little tapas wine bar on Calle cava baja...,casa-lucas-madrid,https://s3-media4.fl.yelpcdn.com/bphoto/xqe6Mv...,False,157,40.412325,-3.709429,"{'address1': 'Calle de la Cava Baja, 30', 'add...",+34 913 65 08 04,€€,None,[],None,None,None,None,None,None,None
6,Bi0S0_Pf_KGuS0oDOkcBQA,Grazie Mille,5,"Calle Mayor, 31, 28013 Madrid, Spain",Italian Pizza,Grazie Mille serves breakfasts that are over-t...,grazie-mille-madrid,https://s3-media4.fl.yelpcdn.com/bphoto/ld7vje...,False,75,40.416110,-3.707440,"{'address1': 'Calle Mayor, 31', 'address2': ''...",+34 913 64 57 85,€€,None,[],None,None,None,None,None,None,None
7,TN6pOyrHDb1Gkvp7zgkYzA,Los Galayos,5,"Calle de Botoneras, 5, 28012 Madrid, Spain",Spanish Tapas Bars Steakhouses,Después de visitar el Palacio Real en Madrid m...,los-galayos-madrid,https://s3-media4.fl.yelpcdn.com/bphoto/F2IXfD...,False,59,40.414860,-3.706690,"{'address1': 'Calle de Botoneras, 5', 'address...",+34 913 66 30 28,€€,None,[],None,None,None,None,None,None,None
8,iNtK9yWv7wOXaPai3uAv-g,InClan Brutal,5,"Calle Álvarez Gato, 4, 28012 Madrid, Spain",Tapas Bars Mediterranean,Amazing cocktails and good service!!!!! Also l...,inclan-brutal-madrid,https://s3-media3.fl.yelpcdn.com/bphoto/qnACZF...,False,171,40.415080,-3.701920,"{'address1': 'Calle Álvarez Gato, 4', 'address...",+34 910 23 80 38,€€,None,[],None,None,None,None,None,None,None
9,F_8sYJoJxI7MXqTD3lWl1g,La Campana,5,"Calle de las Botoneras, 6, 28012 Madrid, Spain",Tapas Bars Spanish Sandwiches,Simple but SO good. There were a few calamari ...,la-campana-madrid-2,https://s3-media1.fl.yelpcdn.com/bphoto/mCKJn1...,False,120,40.414860,-3.706819,"{'address1': 'Calle de las Botoneras, 6', 'add...",+34 913 64 29 84,€,None,[],None,None,Non

### **CHECK RESULTS**

In [10]:
map_madrid = folium.Map(location=[40.427919,-3.680877], zoom_start=14)

for (index, row) in df_distrito.iterrows():
    folium.Marker(location = [row.loc["latitude"], row.loc["longitude"]],
    #popup = row.loc["rotulo"] + " " + row["desc_distrito_local"],
    tooltip = "click").add_to(map_madrid)

map_madrid



In [11]:
df_distrito.to_csv("../data/centro.csv")